## Pose Estimation Models

references:
- stacked hourglass networks for human pose estimation: [https://arxiv.org/abs/1603.06937](https://arxiv.org/abs/1603.06937)
- simple baselines for human pose estimation and tracking: [https://arxiv.org/abs/1804.06208](https://arxiv.org/abs/1804.06208)
- [https://curt-park.github.io/2018-07-03/stacked-hourglass-networks-for-human-pose-estimation/](https://curt-park.github.io/2018-07-03/stacked-hourglass-networks-for-human-pose-estimation/)

code references: 
- [https://github.com/bearpaw/pytorch-pose](https://github.com/bearpaw/pytorch-pose)
- [https://github.com/princeton-vl/pose-hg-train](https://github.com/princeton-vl/pose-hg-train)

dataset annotatation references:
- [https://github.com/bearpaw/pytorch-pose](https://github.com/bearpaw/pytorch-pose)
- [https://github.com/HRNet/HRNet-Human-Pose-Estimation](https://github.com/HRNet/HRNet-Human-Pose-Estimation)

### Stacked Hourglass (2016)

#### Multi-Stage Architecture

<img src="./src_imgs/10.png" width=600><br/>

<img src="./src_imgs/9.jpg"><br/>

Stacked Hourglass는 말그대로 모래시계 모양의 동일한 Network를 여러겹 쌓아서 만든 모델입니다. 이렇게 단일 네트워크를 여러겹 쌓는 구조는 Pose Estimation 분야에서 종종 볼 수 있습니다. 그 이유는 각 Stack마다 heatmap을 출력해보면 알 수 있는데, 2번째 그림과 같이 결과를 refine 시킬 수 있다는 장점을 갖기 때문입니다. 이러한 구조를 Multi-Stage Architecture라 하며 Single-Stage Architecture보다 정밀한 결과를 얻을 것이라는 아이디어에서 출발하였습니다. 

#### Multi-Scale

|![](./src_imgs/6.png)|
|:---:|
|*Hourglass*|

|![](./src_imgs/7.png)|
|:---:|
|*Stacked Hourglass (two-stack)*|

Stacked Hourglass가 포즈를 추출하는 것에 있어 우수한 성능을 내는 또 하나의 이유는 이미지의 모든 scale에서 정보를 얻을 수 있기 때문입니다. 단일 Hourglass network는 high resolution에서 low resolution으로 features를 만들어내는 **Downsampling Process**, 다시 low resolution에서 high resolution으로 resolution을 복구시키는 **Upsampling Process**를 통해 다양한 scale에서 정보를 얻습니다 (e.g. 64->32->16->8->4, 4->8->16->32->64). 

#### Residual Block / Intermediate Supervision

|![](./src_imgs/11.png)|
|:---:|
|*Residual Block*|

|![](./src_imgs/12.png)|
|:---:|
|*Intermediate Supervision*|

Multi-Stage 구조의 가장 큰 문제는 네트워크가 쌓일수록 layer가 깊어져서 Deep Neural Net의 고질적인 문제인 vanishing gradient(깊은 layer에서는 gradient의 영향력이 작아지는 문제)와 degradation(얕은 layer보다 깊은 layer를 갖는 모델이 오히려 성능이 안좋아지는 문제)이 발생한다는 점입니다. 
이것을 Stacked Hourglass는 residual block과 intermediate supervision을 차용함으로 학습 문제를 개선하였습니다. 앞서 Hourglass 구조를 나타내는 그림에서 각 상자는 residual block을 의미합니다.

Residual block의 특징은 module의 input과 output(convolution + activation function 결과)을 element-wise addition한다는 것입니다. 이러한 구조는 gradient를 유지하기 쉽게 만들어주어 vanishing gradient가 발생하기 어렵게 만든다는 장점이 있습니다.

Intermediate supervision이란 각 stack 끝마다 loss layer를 추가하는 것으로 gradient를 비효율적이지만 효과적으로 전달하는 방법입니다. 깊은 stack일수록 gradient의 값이 0에 가깝다는 문제를 직접 각 stack에 gradient를 전달하는 것으로 해결하였습니다.


#### 모델 정의 및 구조 시각화

이제 Hourglass 모델을 정의하고 stack=2 만큼 쌓은 네트워크의 구조를 시각화하겠습니다.

Hourglass 모델을 이루는 기본 block은 앞서 말했듯이 Residual block을 사용합니다.<br/>
Block의 입력인 x와 컨볼루션 결과인 out이 함께 더해져서 연결되는 모습(skip connection)을 확인할 수 있습니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# residual block
class Bottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()

        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=True)
        self.bn3 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 2, kernel_size=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)

        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)

        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out
        

위에서 정의한 residual block은 다음 hourglass module에서 기본 block으로 사용됩니다. 

hourglass에선 다양한 scale의 resolution에서 정보를 얻기 위해 MaxPooling을 이용해 downsampling 하고, 최근접 이웃보간을 통해 upsampling 합니다 (`F.max_pool2d()`, `F.interpolate()`).

In [2]:
class Hourglass(nn.Module):
    def __init__(self, block, num_blocks, planes, depth):
        super(Hourglass, self).__init__()
        self.depth = depth
        self.block = block
        self.hg = self._make_hour_glass(block, num_blocks, planes, depth)

    def _make_residual(self, block, num_blocks, planes):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(planes*block.expansion, planes))
        return nn.Sequential(*layers)

    def _make_hour_glass(self, block, num_blocks, planes, depth):
        hg = []
        for i in range(depth):
            res = []
            for j in range(3):
                res.append(self._make_residual(block, num_blocks, planes))
            if i == 0:
                res.append(self._make_residual(block, num_blocks, planes))
            hg.append(nn.ModuleList(res))
        return nn.ModuleList(hg)

    def _hour_glass_forward(self, n, x):
        up1 = self.hg[n-1][0](x)
        low1 = F.max_pool2d(x, 2, stride=2)
        low1 = self.hg[n-1][1](low1)

        if n > 1:
            low2 = self._hour_glass_forward(n-1, low1)
        else:
            low2 = self.hg[n-1][3](low1)
        low3 = self.hg[n-1][2](low2)
        up2 = F.interpolate(low3, scale_factor=2)
        out = up1 + up2
        return out

    def forward(self, x):
        return self._hour_glass_forward(self.depth, x)
        

이제 Hourglass 모듈을 stack으로 쌓을 수 있는 네트워크를 정의합니다.

`HourglassNet`은 hourglass 스택 개수(`num_stack`), 한 block에 포함된 residual block 개수(`num_block`), parts 개수(`num_classes`)를 조정할 수 있습니다. 

In [3]:
 class HourglassNet(nn.Module):
    '''Hourglass model from Newell et al ECCV 2016'''
    def __init__(self, block, num_stacks=2, num_blocks=4, num_classes=16):
        super(HourglassNet, self).__init__()

        self.inplanes = 64
        self.num_feats = 128
        self.num_stacks = num_stacks
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=True)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_residual(block, self.inplanes, 1)
        self.layer2 = self._make_residual(block, self.inplanes, 1)
        self.layer3 = self._make_residual(block, self.num_feats, 1)
        self.maxpool = nn.MaxPool2d(2, stride=2)

        # build hourglass modules
        ch = self.num_feats*block.expansion
        hg, res, fc, score, fc_, score_ = [], [], [], [], [], []
        for i in range(num_stacks):
            hg.append(Hourglass(block, num_blocks, self.num_feats, 4))
            res.append(self._make_residual(block, self.num_feats, num_blocks))
            fc.append(self._make_fc(ch, ch))
            score.append(nn.Conv2d(ch, num_classes, kernel_size=1, bias=True))
            if i < num_stacks-1:
                fc_.append(nn.Conv2d(ch, ch, kernel_size=1, bias=True))
                score_.append(nn.Conv2d(num_classes, ch, kernel_size=1, bias=True))
        self.hg = nn.ModuleList(hg)
        self.res = nn.ModuleList(res)
        self.fc = nn.ModuleList(fc)
        self.score = nn.ModuleList(score)
        self.fc_ = nn.ModuleList(fc_)
        self.score_ = nn.ModuleList(score_)

    def _make_residual(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=True),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _make_fc(self, inplanes, outplanes):
        bn = nn.BatchNorm2d(inplanes)
        conv = nn.Conv2d(inplanes, outplanes, kernel_size=1, bias=True)
        return nn.Sequential(
                conv,
                bn,
                self.relu,
            )

    def forward(self, x):
        out = []
        # preprocessing
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.maxpool(x)
        x = self.layer2(x)
        x = self.layer3(x)

        # stacking modules
        for i in range(self.num_stacks):
            y = self.hg[i](x)
            y = self.res[i](y)
            y = self.fc[i](y)
            score = self.score[i](y)
            out.append(score)
            if i < self.num_stacks-1:
                fc_ = self.fc_[i](y)
                score_ = self.score_[i](score)
                x = x + fc_ + score_

        return out
    

Stack=1인 HourglassNet의 구조를 출력해보면 다음과 같습니다. 

레이어가 깊어질수록 output resolution이 128->64->32->16->8->4 까지 줄어들고, 
다시 4->8->16->32->64 까지 증가함을 볼 수 있습니다. 또한, 마지막 레이어에선 heatmap([1, 16, 64, 64])을 출력하고 있습니다. 

In [4]:
from torchsummary import summary

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# hourglass
model = HourglassNet(Bottleneck, num_stacks=1, num_blocks=1, num_classes=16).to(device)

summary(model, input_size=(3, 256, 256), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 128, 128]           9,472
       BatchNorm2d-2          [1, 64, 128, 128]             128
              ReLU-3          [1, 64, 128, 128]               0
              ReLU-4          [1, 64, 128, 128]               0
       BatchNorm2d-5          [1, 64, 128, 128]             128
              ReLU-6          [1, 64, 128, 128]               0
            Conv2d-7          [1, 64, 128, 128]           4,160
       BatchNorm2d-8          [1, 64, 128, 128]             128
              ReLU-9          [1, 64, 128, 128]               0
           Conv2d-10          [1, 64, 128, 128]          36,928
      BatchNorm2d-11          [1, 64, 128, 128]             128
             ReLU-12          [1, 64, 128, 128]               0
           Conv2d-13         [1, 128, 128, 128]           8,320
           Conv2d-14         [1, 128, 1

In [5]:
# 2 stacked hourglass
model = HourglassNet(Bottleneck, num_stacks=2, num_blocks=1, num_classes=16).to(device)

summary(model, input_size=(3, 256, 256), batch_size=1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 128, 128]           9,472
       BatchNorm2d-2          [1, 64, 128, 128]             128
              ReLU-3          [1, 64, 128, 128]               0
              ReLU-4          [1, 64, 128, 128]               0
              ReLU-5          [1, 64, 128, 128]               0
       BatchNorm2d-6          [1, 64, 128, 128]             128
              ReLU-7          [1, 64, 128, 128]               0
            Conv2d-8          [1, 64, 128, 128]           4,160
       BatchNorm2d-9          [1, 64, 128, 128]             128
             ReLU-10          [1, 64, 128, 128]               0
           Conv2d-11          [1, 64, 128, 128]          36,928
      BatchNorm2d-12          [1, 64, 128, 128]             128
             ReLU-13          [1, 64, 128, 128]               0
           Conv2d-14         [1, 128, 1

### Simple Baselines (2018)

